In [99]:
import pandas as pd
import os
import arxiv

import urllib
from sklearn.metrics.pairwise import cosine_similarity
import fitz
import tensorflow_hub as hub
import numpy as np 


In [100]:
from function import *

In [101]:
import time
import matplotlib as plt

In [417]:
import re
import sys


In [103]:
#use tranfer learning
#takes longer to load than dan
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [104]:
table = pd.read_csv('combined_table.csv').drop('Unnamed: 0', axis =1)
# test = table.head() #for testing purposes

C:\Users\20193635\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



In [105]:
def get_text(doc):
    blocks = []
    for page in doc.pages():
        b_p = page.get_text('blocks')
        for b in b_p:
            if b[6] == 0: #only text not metadata pictures.
                blocks.append(b[4])

In [422]:
def pdf_similarity(embed, tab_old, result=None, max_iter=1, r_state = 2):
    """
    Wanted statistics?: number of words, similarity, number of pages, percentage of words exactly the same,
    in = table
    """ 
    # for each random row in table, replace=False all rows only sampled once
    sample = tab_old.sample(n=max_iter, replace=False, random_state=r_state)
    not_sample = tab_old.drop(sample.index)
    
    #download two papers from table
    loc_name_Arxiv = 'pdf sim/Arxiv.pdf'
    loc_name_Anth = 'pdf sim/Anth.pdf'
    
    #needs to be changed
    sample[['Succes', 'w_Anth', 'w_Arxiv', 'pages_Anth', 'pages_Arxiv', 'cosine', 'ref_Anth', 'ref_Arxiv', 'Jaccard' ]] =np.nan
    
    
    for index,series in sample.iterrows():
        sample.loc[[index],['Succes']]=0

#         if True:
        #open paper in jupiter/blocks
        try: #sometimes download not available. Then download works but does not give a good pdf so reading does not work
            urllib.request.urlretrieve(series['pdf_url_Arxiv'], loc_name_Arxiv)
            urllib.request.urlretrieve(series['url']+'.pdf', loc_name_Anth)
            print('start with two papers')
            doc_Arxiv = fitz.open(loc_name_Arxiv)
            doc_Anth = fitz.open(loc_name_Anth)

            b_Anth = get_blocks(doc_Anth)
            b_Arxiv = get_blocks(doc_Arxiv)
            
            #try removing sidebar from arxiv
            # if we DO find something
            if [i for i, x in enumerate(b_Arxiv) if x.find('arXiv:')>= 0] !=[]:
                i_sidebar = [i for i, x in enumerate(b_Arxiv) if x.find('arXiv:')>= 0][0]
                if len(b_Arxiv[i_sidebar])< 50: #dont remove references by accident, normal sidebar is length 41
                    x = b_Arxiv.pop(i_sidebar)
#                     print(x)
            
            #remove footnote from anthology (could/did not remove pagenumbers)
            if b_Anth[0][0:18] == 'Proceedings of the' and len(b_Anth[0])<150:
                b_Anth=b_Anth[1:]

#             elem_both_list = set()
            elem_both_list = set(b_Anth)&set(b_Arxiv)
            #sometimes same occurs twice in paper then remove this from elem_both_list
            dup = [x for x in elem_both_list if b_Anth.count(x)>1]
            dup = set(dup + [x for x in elem_both_list if b_Anth.count(x)>1])
            for el in dup:
                elem_both_list.remove(el)
                
        
            #number of citations
            if [i for i, x in enumerate(b_Anth) if x.find('References')>= 0] ==[]:
                ref_Anth='NF'
                ref_Arxiv='NF'
            elif [i for i, x in enumerate(b_Anth) if x.find('References')>= 0] ==[]:
                ref_Anth='NF'
                ref_Arxiv='NF'
            else:              
                start_ref_Anth = [i for i, x in enumerate(b_Anth) if x.find('References')>= 0][0]
                start_ref_Arxiv = [i for i, x in enumerate(b_Arxiv) if x.find('References')>= 0][0]
                
  
                amount_pattern = r'(?:[1][89][0-9]{2}[^0-9]|[2][0][012][0-9][^0-9])'
                amount_expr = re.compile(amount_pattern, re.IGNORECASE)
                l = []
                for i in b_Anth[start_ref_Anth:]:
                    l+=(amount_expr.findall(i))
                ref_Anth=len(l)
                l = []
                for i in b_Arxiv[start_ref_Anth:]:
                    l+=(amount_expr.findall(i))
                ref_Arxiv=len(l)
    
#                 ref_Anth,ref_Arxiv = max([len(b_Anth[start_ref_Anth:]),len(b_Arxiv[start_ref_Arxiv:])],
#                                          [sum([x.count('[') for x in b_Anth[start_ref_Anth:]]),
#                                           sum([x.count('[') for x in b_Arxiv[start_ref_Arxiv:]])])
                
    
            #similarity
            cosine_avg, w_Anth, w_Arxiv, jacc = similarity(b_Anth,b_Arxiv,embedding=embed)
            
            # add statistics to table 'sample'
            sample.loc[[index],['cosine_avg', 'w_Anth', 'w_Arxiv', 'ref_Anth', 'ref_Arxiv', 'Jaccard']]= cosine_avg, w_Anth, w_Arxiv, ref_Anth, ref_Arxiv, jacc
            sample.loc[[index],['pages_Anth', 'pages_Arxiv']] = [doc_Anth.page_count, doc_Arxiv.page_count]
            
            sample.loc[[index],['Succes']]=1
            

        except: #error with downloading 
            print('Exception found some error')
            next 
              
        time.sleep(3)
        
    #end loop
    # join tables
    if type(result) ==pd.DataFrame:
        sample = sample.merge(result,how='outer')
    return not_sample, sample

In [426]:
def similarity(anth, arxiv, embedding):
#     print('start sim')
    anth = [clean_n(x) for x in anth]
    arxiv = [clean_n(x) for x in arxiv]
    
    w_anth = " ".join(anth).split()
    w_arxiv = " ".join(arxiv).split()
    
    set_Anth = set(w_anth)
    set_Arxiv = set(w_arxiv)
    
    # jacc = Intersection/Union
    jacc = len(set_Anth & set_Arxiv)/len(set_Anth | set_Arxiv)
    
    df = pd.DataFrame()
    df['Anth'] = [" ".join(list(x)) for x in np.array_split(np.array(w_anth),20)]
    df['Arxiv'] = [" ".join(list(x)) for x in np.array_split(np.array(w_arxiv),20)]
    
    df['words Anth'] = df['Anth'].apply(lambda x: len(x.split()))
    df['words Arxiv'] = df['Arxiv'].apply(lambda x: len(x.split()))
    
    df['cosine'] = df.apply(lambda x:cosine_similarity(embedding([x.Anth]),embedding([x.Arxiv])), axis=1)
    df['cosine'] = df['cosine'].astype(float)

#     display(df)
#     df.tolatex()

    #nr of words per paper
    nr_w_Anth = sum(df['words Anth'])
    nr_w_Arxiv = sum(df['words Arxiv'])

    cosine_avg = round(np.average(df['cosine']),6)
    
    return cosine_avg, nr_w_Anth, nr_w_Arxiv, jacc 


In [ ]:
table.head()

In [234]:
tab = table[3:6]

In [235]:
tab2 = prepare_table(tab)

In [236]:
tab2

,title,year,month,url,title_Arxiv,pdf_url_Arxiv,published_Arxiv,result_Arxiv
3,HateBERT: Retraining BERT for Abusive Language...,2021,August,https://aclanthology.org/2021.woah-1.3,HateBERT: Retraining BERT for Abusive Language...,http://arxiv.org/pdf/2010.12472v1,2020-10-23 15:14:14+00:00,http://arxiv.org/abs/2010.12472v2
4,Memes in the Wild: Assessing the Generalizabil...,2021,August,https://aclanthology.org/2021.woah-1.4,Memes in the Wild: Assessing the Generalizabil...,http://arxiv.org/pdf/2107.04313v1,2021-07-09 09:04:05+00:00,http://arxiv.org/abs/2107.04313v1
5,Measuring and Improving Model-Moderator Collab...,2021,August,https://aclanthology.org/2021.woah-1.5,Measuring and Improving Model-Moderator Collab...,http://arxiv.org/pdf/2107.04212v1,2021-07-09 05:07:25+00:00,http://arxiv.org/abs/2107.04212v1


In [427]:
start = time.time()
not_sample, sample = run(embed, tab2,max_iter = 1)
end = time.time()
# print("Time taken:            {:.3f}s".format(end - start))   
print("Time taken:      {:.3f}s".format(end - start), file=sys.stderr)

start with two papers


Time taken:      16.661s


In [428]:
# sample.columns
sample

,title,year,month,url,title_Arxiv,pdf_url_Arxiv,published_Arxiv,result_Arxiv,Succes,w_Anth,w_Arxiv,pages_Anth,pages_Arxiv,cosine,ref_Anth,ref_Arxiv,Jaccard,cosine_avg
5,Measuring and Improving Model-Moderator Collab...,2021,August,https://aclanthology.org/2021.woah-1.5,Measuring and Improving Model-Moderator Collab...,http://arxiv.org/pdf/2107.04212v1,2021-07-09 05:07:25+00:00,http://arxiv.org/abs/2107.04212v1,1.0,10041.0,10017.0,18.0,18.0,NaN,70.0,78.0,0.968085,0.977536


# test on random papers

In [256]:
ran = pd.read_csv('ran_sample.csv').drop('Unnamed: 0', axis =1)

In [258]:
table_pred = prepare_table(table)
cop = table_pred.copy()
cop = cop.reset_index()
# ran = table_pred.head(10)#['title', 'year', 'month', 'url'] #[5:10]#['title', 'year', 'month', 'url']]
# ran2 = table_pred.tail(10)
ran = cop.loc[:9,['title', 'url']]
ran2 = cop.loc[len(table_pred)-10:len(table_pred),['title_Arxiv', 'pdf_url_Arxiv', 'result_Arxiv']]
ran2 = ran2.reset_index()
random = ran.join(ran2)
# random.loc[[3],['title_Arxiv', 'pdf_url_Arxiv', 'published_Arxiv', 'result_Arxiv']] = ran2

In [265]:
random

,title,url,index,title_Arxiv,pdf_url_Arxiv,result_Arxiv
0,Modeling Profanity and Hate Speech in Social M...,https://aclanthology.org/2021.woah-1.2,8875,Overcoming the Curse of Sentence Length for Ne...,http://arxiv.org/pdf/1409.1257v1,http://arxiv.org/abs/1409.1257v2
1,HateBERT: Retraining BERT for Abusive Language...,https://aclanthology.org/2021.woah-1.3,8876,A Novel Two-stage Framework for Extracting Opi...,http://arxiv.org/pdf/2101.09743v1,http://arxiv.org/abs/2101.09743v1
2,Memes in the Wild: Assessing the Generalizabil...,https://aclanthology.org/2021.woah-1.4,8877,Arabic Spelling Correction using Supervised Le...,http://arxiv.org/pdf/1409.8309v1,http://arxiv.org/abs/1409.8309v1
3,Measuring and Improving Model-Moderator Collab...,https://aclanthology.org/2021.woah-1.5,8878,Arabizi Detection and Conversion to Arabic,http://arxiv.org/pdf/1306.6755v1,http://arxiv.org/abs/1306.6755v1
4,Improving Counterfactual Generation for Fair H...,https://aclanthology.org/2021.woah-1.10,8879,DiscoTK: Using Discourse Structure for Machine...,http://arxiv.org/pdf/1911.12547v1,http://arxiv.org/abs/1911.12547v1
5,Mitigating Biases in Toxic Language Detection ...,https://aclanthology.org/2021.woah-1.12,8880,Credibility Adjusted Term Frequency: A Supervi...,http://arxiv.org/pdf/1405.3518v1,http://arxiv.org/abs/1405.3518v2
6,When the Echo Chamber Shatters: Examining the ...,https://aclanthology.org/2021.woah-1.18,8881,Improving Agreement and Disagreement Identific...,http://arxiv.org/pdf/1606.05706v1,http://arxiv.org/abs/1606.05706v1
7,Text Simplification for Comprehension-based Qu...,https://aclanthology.org/2021.wnut-1.1,8882,Finding Eyewitness Tweets During Crises,http://arxiv.org/pdf/1403.1773v1,http://arxiv.org/abs/1403.1773v1
8,Detecting Depression in Thai Blog Posts: a Dat...,https://aclanthology.org/2021.wnut-1.3,8883,Temporal Analysis of Language through Neural L...,http://arxiv.org/pdf/1405.3515v1,http://arxiv.org/abs/1405.3515v1
9,Can images help recognize entities? A study of...,https://aclanthology.org/2021.wnut-1.11,8884,A Deep Architecture for Semantic Parsing,http://arxiv.org/pdf/1404.7296v1,http://arxiv.org/abs/1404.7296v1


In [276]:
start = time.time()
r_not_sample, r_sample = run(embed, random,max_iter = 10)
end = time.time()
# print("Time taken:            {:.3f}s".format(end - start))   
print("Time taken:      {:.3f}s".format(end - start), file=sys.stderr)

start with two papers
start with two papers
start with two papers
start with two papers
start with two papers
start with two papers
start with two papers
start with two papers
start with two papers
start with two papers


Time taken:      280.505s


In [267]:
r_sample #new 20

,title,url,index,title_Arxiv,pdf_url_Arxiv,result_Arxiv,Succes,w_Anth,w_Arxiv,w_both,pages_Anth,pages_Arxiv,cosine,len_blocks,len_Anth,len_Arxiv,len_both,ref_Anth,ref_Arxiv,Jaccard
4,Improving Counterfactual Generation for Fair H...,https://aclanthology.org/2021.woah-1.10,8879,DiscoTK: Using Discourse Structure for Machine...,http://arxiv.org/pdf/1911.12547v1,http://arxiv.org/abs/1911.12547v1,1.0,5436.0,4296.0,NaN,10.0,8.0,0.490809,NaN,NaN,NaN,NaN,75.0,22.0,0.095070
1,HateBERT: Retraining BERT for Abusive Language...,https://aclanthology.org/2021.woah-1.3,8876,A Novel Two-stage Framework for Extracting Opi...,http://arxiv.org/pdf/2101.09743v1,http://arxiv.org/abs/2101.09743v1,1.0,4919.0,5728.0,NaN,9.0,9.0,0.492599,NaN,NaN,NaN,NaN,65.0,26.0,0.106205
5,Mitigating Biases in Toxic Language Detection ...,https://aclanthology.org/2021.woah-1.12,8880,Credibility Adjusted Term Frequency: A Supervi...,http://arxiv.org/pdf/1405.3518v1,http://arxiv.org/abs/1405.3518v2,1.0,4248.0,2542.0,NaN,7.0,5.0,0.393532,NaN,NaN,NaN,NaN,36.0,22.0,0.096110
0,Modeling Profanity and Hate Speech in Social M...,https://aclanthology.org/2021.woah-1.2,8875,Overcoming the Curse of Sentence Length for Ne...,http://arxiv.org/pdf/1409.1257v1,http://arxiv.org/abs/1409.1257v2,1.0,7037.0,4359.0,NaN,11.0,7.0,0.501211,NaN,NaN,NaN,NaN,28.0,57.0,0.105679
7,Text Simplification for Comprehension-based Qu...,https://aclanthology.org/2021.wnut-1.1,8882,Finding Eyewitness Tweets During Crises,http://arxiv.org/pdf/1403.1773v1,http://arxiv.org/abs/1403.1773v1,1.0,5292.0,6346.0,NaN,10.0,11.0,0.529852,NaN,NaN,NaN,NaN,40.0,23.0,0.115042
2,Memes in the Wild: Assessing the Generalizabil...,https://aclanthology.org/2021.woah-1.4,8877,Arabic Spelling Correction using Supervised Le...,http://arxiv.org/pdf/1409.8309v1,http://arxiv.org/abs/1409.8309v1,1.0,4865.0,3723.0,NaN,10.0,6.0,0.378114,NaN,NaN,NaN,NaN,87.0,16.0,0.104698
3,Measuring and Improving Model-Moderator Collab...,https://aclanthology.org/2021.woah-1.5,8878,Arabizi Detection and Conversion to Arabic,http://arxiv.org/pdf/1306.6755v1,http://arxiv.org/abs/1306.6755v1,1.0,10061.0,4631.0,NaN,18.0,8.0,0.450464,NaN,NaN,NaN,NaN,405.0,3.0,0.097291
6,When the Echo Chamber Shatters: Examining the ...,https://aclanthology.org/2021.woah-1.18,8881,Improving Agreement and Disagreement Identific...,http://arxiv.org/pdf/1606.05706v1,http://arxiv.org/abs/1606.05706v1,1.0,8666.0,6637.0,NaN,15.0,10.0,0.521938,NaN,NaN,NaN,NaN,78.0,35.0,0.114820
9,Can images help recognize entities? A study of...,https://aclanthology.org/2021.wnut-1.11,8884,A Deep Architecture for Semantic Parsing,http://arxiv.org/pdf/1404.7296v1,http://arxiv.org/abs/1404.7296v1,1.0,6047.0,3070.0,NaN,10.0,6.0,0.478675,NaN,NaN,NaN,NaN,27.0,24.0,0.120555
8,Detecting Depression in Thai Blog Posts: a Dat...,https://aclanthology.org/2021.wnut-1.3,8883,Temporal Analysis of Language through Neural L...,http://arxiv.org/pdf/1405.3515v1,http://arxiv.org/abs/1405.3515v1,1.0,3460.0,2732.0,NaN,6.0,5.0,0.326489,NaN,NaN,NaN,NaN,37.0,17.0,0.110707
